In [ ]:
%jsroot off

In [ ]:
#include<TFile.h>
#include<TH1.h>
#include<TMath.h>
#include<math.h>
#include<TF1.h>
#include<TLine.h>
#include<TStyle.h>
#include<TPoint.h>
#include<iostream>
#include<string>
#include<stdio.h>

In [ ]:
// Selección de fit:
const Int_t fit = 1; // Opciones: 1: Gauss. 2: Breit Wigner.

In [ ]:
void PrintCanvas(TCanvas* c1, string title) {
    string outFolder="Optimization";
    string tpng = outFolder+"/"+title+".png";
    c1->SaveAs(tpng.c_str());

    string tpdf = outFolder+"/"+title+".pdf";
    c1->SaveAs(tpdf.c_str());

    string troot = outFolder+"/"+title+".root";
    c1->SaveAs(troot.c_str());
    
    return;
}

In [ ]:
TFile *f = new TFile("//Plotting/histograms/mass_four_lep.root","READ");
//f->ls();
TH1F * h1 = (TH1F*)f->Get("c");

h1->SaveAs("Optimization/mass_four_lep.png");
h1->SaveAs("Optimization/mass_four_lep.root");

h1->Draw();

In [ ]:
TCanvas *c1 = new TCanvas("c1","c1",900, 600); //este es comodin, por si queremos dibujar algo
/*TCanvas *cToys = new TCanvas("cToys","cToys",900, 600); //este lo usaremos para los modelos
cToys->Divide(2,1); //lo dividimos en una red the 3x1*/

In [ ]:
c1->cd();
//c1->SetGrid();
/*gStyle->SetPaperSize(20,26);

gStyle->SetPadTopMargin(0.05);
gStyle->SetPadRightMargin(0.05); 
gStyle->SetPadBottomMargin(0.16);
gStyle->SetPadLeftMargin(0.16);*/

gStyle->SetTitleXOffset(1.5);
gStyle->SetTitleYOffset(1.5);

//Legendas
gStyle->SetOptTitle (0/1);
gStyle->SetOptStat(0);
gStyle->SetOptFit(1100);

gStyle->SetLegendBorderSize(0);
gStyle->SetStatFontSize(0.02);
gStyle->SetStatW(0.15);
gStyle->SetStatBorderSize(0);
gStyle->SetStatX(.39); gStyle->SetStatY(.81); // Posición

gStyle->SetPalette(1); //opcional: cambia la paleta de colores en los plots.

In [ ]:
/*// create the rest plot
data->Add(bgd,-1);
//h_ratio->GetYaxis()->SetTitle("Data - bkg      ");

float errore;
int nbine = data[fIter->first]->GetNbinsX();
for(int i_bin=0;i_bin<=nbine;i_bin++){
        errore = data[fIter->first]->GetBinError(i_bin);
        h_ratio->SetBinError(i_bin, errore);
} 

TH1F *copy_fit = new TH1F("copy_fit","copy_fit",5500,105,160); //  smooth
copy_fit->Eval(fit);
copy_fit->SetLineColor(2);
copy_fit->SetLineStyle(1);
copy_fit->SetLineWidth(2);

h_ratio2 = (TH1F*)copy_fit->Clone("h_ratio2");
h_ratio2->Add(bgd,-1);
delete copy_fit;*/

In [ ]:
Float_t minVal = 110.0; //Valor mínimo del intervalo.
Float_t maxVal = 135.0; //Valor máximo del intervalo.

In [ ]:
Int_t n = 7;
Int_t nbins = 7;
TH1D *data = new TH1D("data","data",n,minVal,maxVal); // Histograma de datos.

// Con 7 bins:
if (n == 7) {  
    data->SetBinContent(1, 1);
    data->SetBinContent(2, 0);
    data->SetBinContent(3, 1);
    data->SetBinContent(4, 8);
    data->SetBinContent(5, 4);
    data->SetBinContent(6, 0);
    data->SetBinContent(7, 2);
}
data->SetName("data");
data->GetXaxis()->SetTitle("m_{4l} [GeV]");
c1->SetBottomMargin(0.12);
data->GetYaxis()->SetTitle("Events/5 GeV");
data->SetAxisRange(0,11,"y");
data->SetLineColor(kBlack);
data->SetLineWidth(1);
data->SetMarkerStyle(21);
data->SetMarkerSize(1);
data->SetMarkerColor(kBlack);

In [ ]:
if (fit == 1) {
    //PDF de la señal, más info: https://root.cern.ch/root/html524/TMath.html#TMath:Gaus
    TF1 *signalModel = new TF1("signalModel","[2]*TMath::Gaus(x,[0],[1],1)",minVal,maxVal);
    signalModel->SetParNames ("mean","sigma","constant"); //nombres de los parámetros
    signalModel->SetParameters(125.0,5.0,1.0); //valores de los parámetros
    //signalModel->FixParameter(0,123.8);
    signalModel->SetLineColor(kBlack);
    signalModel->SetLineWidth(2);
}
Float_t normSig;
if (fit == 2) {
    //PDF de la señal, más info: https://root.cern.ch/root/html524/TMath.html#TMath:BreitWigner
    TF1 *signalModel = new TF1("signalModel","[2]*TMath::BreitWigner(x,[0],[1])",minVal,maxVal);
    signalModel->SetParNames("mass","width","constant"); //nombres de los parámetros
    signalModel->SetParameters(125.0,5.0,1.0); //valores de los parámetros
    //signalModel->FixParameter(0,123.8);
    signalModel->SetLineColor(kBlack);
    signalModel->SetLineWidth(2);
}

In [ ]:
r = data->Fit("signalModel",  "LQS", "", minVal, maxVal); // L: likelihood, Q: quiet, reduce printing, S: save fit result
r->Print("v");
//r->Print();
//cout << "Probabilidad de = " << TMath::Prob(signalModel->MinFcnValue()) << endl;

In [ ]:
cout << "Probabilidad de X2= " << TMath::Prob(signalModel->GetChisquare(),4) << endl;

In [ ]:
TGraphErrors *gr = new TGraphErrors(data);
Int_t nPoints = data->GetNbinsX();
for (Int_t i=0; i<nPoints; i++){
    gr->SetPoint(i, data->GetBinCenter(i+data->FindBin(110)), 0); 
}
(TVirtualFitter::GetFitter())->GetConfidenceIntervals(gr,0.68);
gr->SetName("gr");
gr->SetFillColor(3);
data->Draw("E");
gr->Draw("E3");
data->Draw("E1P SAME");

In [ ]:
//create legend
auto legend = new TLegend(.63,.64,.90,.87);
legend->AddEntry("data","data","lep");
if (fit == 1) legend->AddEntry(signalModel,"Gaussian fit","L");
if (fit == 2) legend->AddEntry(signalModel,"Breit Wigner fit","L");
legend->AddEntry("gr","#pm 1 #sigma","f");
legend->Draw("SAME");
c1->Draw();
PrintCanvas(c1, "fit"); // Guardar fit en formato PNG, PDF y ROOT.

Primero creamos un histograma bidimensional

In [ ]:
Float_t stepMass = 0.5;
Float_t stepWidth = 0.25;
binMass = int((maxVal-minVal)/stepMass);
binWidth = int(10./stepWidth);
L_Plot = new TH2D("L_Plot","L_Plot",binMass,minVal-stepMass/2.0,maxVal-stepMass/2.0,binWidth,0.5,10.5);

//cout << "binMass: " << binMass << endl;
//cout << "binWidth: " << binWidth << endl;

Luego realizamos ajustes fijando tanto la masa como la anchura, de forma que nos permitan obtener el $\chi^{2}$ en función de dichos parámetros. Sin fijarlos, obtendríamos siempre el valor que más se ajuste a los datos (en el mínimo!).  

In [ ]:
//this is for the likelihood con the mass and width
L_Plot->Clear();
for (Float_t masses = minVal; masses < maxVal; masses+=stepMass){
    //cout << "Filling: externo mass: "<< masses << ". bin:" << L_Plot->GetXaxis()->FindBin(masses) << endl;
    for (Float_t widths = 0.25; widths < 10.5; widths+=stepWidth){
        //cout << "Filling: interno mass and bin: " << widths << L_Plot->GetYaxis()->FindBin(widths) << endl;
        
        for (Int_t i=0; i<3; i++) signalModel->ReleaseParameter(i);
        signalModel->FixParameter(0,masses);
        signalModel->FixParameter(1,widths);
        r = data->Fit("signalModel","QLS");
        L_Plot->SetBinContent(L_Plot->FindBin(masses,widths),r->MinFcnValue());
    }
}

In [ ]:
//remove borders..
//L_Plot->SetAxisRange(0.5,10,"y");
//L_Plot->SetAxisRange(0,20,"z");
L_Plot->SetAxisRange(110,135,"x");
L_Plot->GetXaxis()->SetTitle("m_{4l} [GeV]");
L_Plot->GetYaxis()->SetTitle("Width of the signal [GeV]");
c1->SetRightMargin(0.15);
L_Plot->GetZaxis()->SetTitle("-ln(L)");
L_Plot->Draw("colz");
gStyle->SetOptStat(000000);
c1->Draw();
PrintCanvas(c1, "2D_profile_likelihood"); // Guardar 2D_hyperspace en formato PNG, PDF y ROOT.

In [ ]:
auto c2=new TCanvas("Canvas2","Canvas2",900,600);
c2->Divide(2,2);
c2->cd(1); L_Plot->ProjectionX()->Draw("E");
c2->cd(2); L_Plot->ProjectionY()->Draw("E");
c2->cd(3); L_Plot->ProfileX()->Draw("E");
c2->cd(4); L_Plot->ProfileY()->Draw("E");
c2->Draw();

Cuando se realiza una proyección a lo largo de la dirección x(y), para cada ubicación a lo largo del eje x(y), se suman todos los contenidos de la ubicación a lo largo del eje y(x).
Cuando se realiza un perfil a lo largo de la dirección x(y), para cada contenedor a lo largo del eje x(y), el promedio de todos los contenidos del contenedor a lo largo de y(x) se calcula junto con su valor eficaz y se muestra como un símbolo con barra de error.
https://root.cern.ch/root/htmldoc/guides/primer/ROOTPrimer.html

In [ ]:
c1->cd();
// remove borders..
L_Plot->SetAxisRange(0.5,10,"y");
L_Plot->SetAxisRange(115,128,"x");
L_Plot->GetXaxis()->SetTitle("m_{4l} [GeV]");
L_Plot->GetYaxis()->SetTitle("Width of the signal [GeV]");
L_Plot->GetZaxis()->SetTitle("-ln(L)");
L_Plot->Draw("surf2");
gStyle->SetOptStat(000000);
c1->Draw();
PrintCanvas(c1, "3D_profile_likelihood_1"); // Guardar 3D_hyperspace en formato PNG, PDF y ROOT.

In [ ]:
// remove borders..
L_Plot->SetAxisRange(0.5,10,"y");
L_Plot->SetAxisRange(115,128,"x");
L_Plot->GetXaxis()->SetTitle("m_{4l} [GeV]");
L_Plot->GetYaxis()->SetTitle("Width of the signal [GeV]");
L_Plot->GetZaxis()->SetTitle("-ln(L)");
L_Plot->Draw("surf3");
gStyle->SetOptStat(000000);
c1->Draw();
PrintCanvas(c1, "3D_profile_likelihood_2"); // Guardar 3D_hyperspace en formato PNG, PDF y ROOT.